In [0]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [0]:
!kaggle competitions download -c grasp-and-lift-eeg-detection

100% 915M/915M [00:08<00:00, 104MB/s]

 97% 5.00M/5.16M [00:00<00:00, 27.5MB/s]
100% 5.16M/5.16M [00:00<00:00, 25.2MB/s]
 96% 146M/153M [00:01<00:00, 104MB/s]
100% 153M/153M [00:01<00:00, 105MB/s]


In [0]:
!unzip train.zip
!unzip test.zip

Archive:  train.zip
   creating: train/
  inflating: train/subj10_series1_data.csv  
  inflating: train/subj10_series1_events.csv  
  inflating: train/subj10_series2_data.csv  
  inflating: train/subj10_series2_events.csv  
  inflating: train/subj10_series3_data.csv  
  inflating: train/subj10_series3_events.csv  
  inflating: train/subj10_series4_data.csv  
  inflating: train/subj10_series4_events.csv  
  inflating: train/subj10_series5_data.csv  
  inflating: train/subj10_series5_events.csv  
  inflating: train/subj10_series6_data.csv  
  inflating: train/subj10_series6_events.csv  
  inflating: train/subj10_series7_data.csv  
  inflating: train/subj10_series7_events.csv  
  inflating: train/subj10_series8_data.csv  
  inflating: train/subj10_series8_events.csv  
  inflating: train/subj11_series1_data.csv  
  inflating: train/subj11_series1_events.csv  
  inflating: train/subj11_series2_data.csv  
  inflating: train/subj11_series2_events.csv  
  inflating: train/subj11_series3_data.c

In [0]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [0]:
def prepare_data_train(fname):
    """ read and prepare training data """
    data = pd.read_csv(fname)
    events_fname = fname.replace('_data','_events')
    labels= pd.read_csv(events_fname)
    clean=data.drop(['id' ], axis=1)#remove id
    labels=labels.drop(['id' ], axis=1)#remove id
    return  clean,labels


scaler= StandardScaler()
def data_preprocess_train(X):
    X_prep=scaler.fit_transform(X)
    
    return X_prep
def data_preprocess_test(X):
    X_prep=scaler.transform(X)
    
    return X_prep

In [0]:
subjects=range(1,2)

In [0]:
def load_data(subjects,series):
  y_raw= []
  raw = []
  for subject in subjects:
    print("Loading from subject ",subject)
    
    for ser in series:
      fname ='train/subj%d_series%d_data.csv' % (subject,ser)
      # print(fname)
      data,labels=prepare_data_train(fname)
      raw.append(data)
      y_raw.append(labels)

    X = pd.concat(raw)
    y = pd.concat(y_raw)
      
    X =np.asarray(X.astype(float))
    y = np.asarray(y.astype(float))
  return X,y

In [0]:
X_train,y_train=load_data(subjects,series =[1,2,5,6,7,8])
X_train=data_preprocess_train(X_train)
print(X_train.shape,y_train.shape)

Loading from subject  1
(1088538, 32) (1088538, 6)


In [0]:
time_steps = 320
subsample = 10
def generator(batch_size):
  while 1:
    x_time_data = np.zeros((batch_size, time_steps//subsample, 32))
    yy = []
    for i in range(batch_size):
      random_index = np.random.randint(0, len(X_train)-time_steps)
      x_time_data[i] = X_train[random_index:random_index+time_steps:subsample]
      yy.append(y_train[random_index + time_steps])
    yy = np.asarray(yy)
    return x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1], x_time_data.shape[2], 1)), yy

In [0]:
x,y=generator(32)
print(x.shape)
print(y.shape)

(32, 32, 32, 1)
(32, 6)
